## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression


%matplotlib inline

In [2]:
df = pd.read_csv('./chronic_kidney_disease_full.csv')


### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

In [3]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [4]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
rbc      248 non-null object
pc       335 non-null object
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
sod      313 non-null float64
pot      312 non-null float64
hemo     348 non-null float64
pcv      329 non-null float64
wbcc     294 non-null float64
rbcc     269 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


I'd like to check out age, blood pressure, red blood cell count, white blood cell count, sodium, and potassium.

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [6]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [7]:
df_full = df.dropna()


In [8]:
df_full.shape

(158, 25)

In [9]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

We'd only have 158 observations left. Getting rid of more than half our case files because of missing data that might not even be interested in is not good Data science. Not just that, but the remaining data we have left would no longer be from 'random' respondents.  

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

1) Any answers might be subject to human error.  

2) Hospitals might get busy and nurses might not have time to get all the answers.

3) A patient might lie.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Just assume everyone does have CKD.  That way you'll never give a false negative.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We maximized sensitivity.  We minimized false negatives, which as pointed out in class are vastly worse than false positives.  

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

We're freaking out a lot of people who don't have CKD and needlessly putting their health insurance through the ringer. 

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Tell everybody that they don't have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimized specificity.  We minimized false positives. 

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

You'd have a lot of people walking around with CKD and not knowing what was wrong with them.  This could seriously endanger a person's life not to mention make you vulnarable to (rightfully) draconian lawsuits.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [10]:
df.drop(['rbc','pc','sod','pot','pcv','wbcc','rbcc'], axis=1, inplace=True)
#just get rid of features that have too many missing oservations to just make assumptions about.  

In [11]:
dfd = df
#copy of original data

In [12]:
dfd.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'pcc', 'ba', 'bgr', 'bu', 'sc', 'hemo',
       'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [13]:
dfd.replace(np.nan, 0, inplace=True)

In [14]:
#I'll just make any values zero for which zero does not imply a dead person, and otherwise give the worst 
#variable a patient could have.  This will increase sensitivity. 

In [15]:
dfd['pcc'].replace(0, 'present', inplace=True)
dfd['ba'].replace(0, 'present', inplace=True)
dfd['htn'].replace(0, 'present', inplace=True)
dfd['dm'].replace(0, 'yes', inplace=True)
dfd['appet'].replace(0, 'poor', inplace=True)
dfd['cad'].replace(0, 'yes', inplace=True)
dfd['pe'].replace(0, 'yes', inplace=True)
dfd['ane'].replace(0, 'yes', inplace=True)

In [16]:
dfd['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [17]:
dfd.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'pcc', 'ba', 'bgr', 'bu', 'sc', 'hemo',
       'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [18]:
dfd = pd.get_dummies(data=dfd, drop_first=True)


In [19]:
dfd.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_poor', 'pe_yes', 'ane_yes', 'class_notckd'],
      dtype='object')

In [20]:
dfd.drop('class_notckd', axis=1, inplace=True)
#our targets are people with ckd

In [21]:
dfd ['class'] = df['class']

In [22]:
dfd.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_poor', 'pe_yes', 'ane_yes', 'class'],
      dtype='object')

In [23]:
col_list = list(dfd.columns)
features = []
[features.append(col) for col in col_list if col != 'class']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [24]:
X = dfd[features]
y = dfd['class']

In [25]:
poly = PolynomialFeatures(include_bias=False)
#if bias were True we'd have rows with features that indicate a dead patient

In [26]:
X_poly = poly.fit_transform(X)
# Human body is complex and I don't have an M.D.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y)

In [28]:
ss = StandardScaler()

In [29]:
ss.fit(X_train)
#Variables are at different scales, so this is an important step.  

StandardScaler(copy=True, with_mean=True, with_std=True)

In [30]:
X_train_s = ss.transform(X_train)
X_test_s = ss.transform(X_test)

In [31]:
logreg = LogisticRegression()

In [32]:
logreg.fit(X_train_s, y_train)

/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
logreg.score(X_train_s, y_train)

0.97

In [34]:
logreg.score(X_test_s, y_test)

0.9

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [35]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logreg.coef_))], axis = 1)
coefficients.sort_values

<bound method DataFrame.sort_values of                0         0
0            age  0.304683
1             bp  0.410498
2             sg -0.041689
3             al -0.617305
4             su -0.145184
5            bgr  0.264947
6             bu  0.073556
7             sc -0.285698
8           hemo -0.343968
9    pcc_present  0.017992
10    ba_present  0.133996
11   htn_present  0.164390
12       htn_yes -0.229348
13        dm_yes -0.310415
14       cad_yes -0.023242
15    appet_poor -0.291595
16        pe_yes -0.193755
17       ane_yes -0.178015
18           NaN -0.523162
19           NaN -0.472170
20           NaN  0.297277
21           NaN -0.332149
22           NaN -0.011754
23           NaN -0.379415
24           NaN -0.016732
25           NaN -0.251455
26           NaN  0.610173
27           NaN -0.081459
28           NaN -0.018792
29           NaN  0.164328
..           ...       ...
159          NaN  0.012210
160          NaN  0.005821
161          NaN  0.164390
162          NaN

The blood pressure coefficient of 0.294711 implies that as your blood pressure gets higher your risk of CKD increases.

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

The age coefficient of 0.456884 implies that a respondent is likely to have CKD as they age.  But this seems a little high so I'm suspecting that I did something wrong.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

First, by their nature regression models predict contious variables.  Second, as the score would indicate the explained variability is very high which is good when you want to optimize sensitivity.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

It makes sense because all our test data is actually our training data.  The real test data is from patients we haven't seen yet. 

In [36]:
predictions = logreg.predict(X_test_s)

In [37]:
con = confusion_matrix(y_test, predictions)

In [38]:
con = pd.DataFrame(con, columns=['Predicted Negative','Predicted Positive'], index=['True Negative','True Positive'])

In [39]:
con

,Predicted Negative,Predicted Positive
True Negative,50,7
True Positive,3,40


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

In the case of medicine we should optimize sensitivity.  It's better to be safe than sorry, so we should risk getting more false positives than false negatives. If someone gets a false positive the worst that happens is that they spend a few hours undergoing tests just take make sure that they don't have a life-threatening illness.  If someone gets a false negative then someone will walk out of a hospital thinking that they are fine but in reality need medical attention.  The former is vastly preferable.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Unbalanced classes are generally a problem because the minority class is at risk of not contributing to the training data enough to generate accurate predicionts that apply to them. Therefore our model isn't useful in predicting if they are sick or not.  In this particular case it isn't much of a problem because people who don't have CKD make up 38 % of the respondents. So while they are in the minority they still have influence on the models predictive power.
But to even things out and increase our sensitivity further we could oversample from the class that doesn't have CKD.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

If we only got data entireley from people who do have CKD or don't have CKD, we wouldn't know what statistically distinguishes the two groups and our model wouldn't be reliable.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [40]:
dfd.shape

(400, 19)

In [41]:
from sklearn.utils import resample

In [42]:
dfd_maj = dfd[ dfd['class'] == 'ckd' ]
dfd_min = dfd[ dfd['class'] == 'notckd' ]
#seperate the classes

In [43]:
dfd_maj.shape

(250, 19)

In [44]:
df_upsample = resample(dfd_maj, replace = True, n_samples = 200000)

In [45]:
df_upsample = pd.concat([df_upsample, dfd_min])

In [46]:
df_upsample['class'].value_counts()

ckd       200000
notckd       150
Name: class, dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [47]:
X = df_upsample[features]
y = df_upsample['class']

In [48]:
X_poly = poly.fit_transform(X)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y)

In [50]:
ss.fit(X_train)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [51]:
X_train_sb = ss.transform(X_train)
X_test_sb = ss.transform(X_test)

In [52]:
logreg.fit(X_train_sb, y_train)

/Users/eliregen/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
logreg.score(X_train_sb, y_train)

0.9997268706032829

In [54]:
logreg.score(X_test_sb, y_test)


0.9996003037691354

In [55]:
predictions = logreg.predict(X_test_sb)

In [56]:
cm_boot = confusion_matrix(y_test, predictions)

In [57]:
cm_boot = pd.DataFrame(cm_boot, columns=['Predicted Negative','Predicted Positive'], 
                       index=['Actual Negative','Actual Positive'])

In [58]:
cm_boot


,Predicted Negative,Predicted Positive
Actual Negative,49995,0
Actual Positive,20,23


Now there aren't any false positives.  

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!